In [ ]:
import json
import itertools
import sys
from collections import defaultdict

In [ ]:
challenges = json.load(open("challenges.json"))

for c in challenges:
    c["champions"] = set(c["champions"])
    c["qte"] = int(c["qte"])
    
champions = set()
for c in challenges:
    champions |= c["champions"]
    
print(len(champions))

challenges_by_champions = defaultdict(list)
for challenge in challenges:
    for champion in challenge["champions"]:
        challenges_by_champions[champion].append(challenge["challenge_name"])
challenges_by_champions = dict(challenges_by_champions)

champions_by_challenge = {}
for challenge in challenges:
    champions_by_challenge[challenge["challenge_name"]] = challenge["champions"]

factions = {challenge["challenge_name"] for challenge in challenges if challenge["qte"] == 5}
synergies = {challenge["challenge_name"] for challenge in challenges if challenge["qte"] == 3}

print(factions)


In [ ]:
def find_challenges(comp):
    chall = set()
    for c in challenges:
        if len(c["champions"].intersection(comp)) >= c["qte"]:
            chall.add(c["challenge_name"])
    return chall


find_challenges(set(["Ahri", "Akali", "Shen", "Yasuo", "Zed"]))

In [ ]:
def find_comp(champions_, threshold_min=0, threshold_max=sys.maxsize, max_depth=1e7):
    # sort them by number of challenges to find good match first
    champions_ = list(champions_)
    champions_.sort(key=lambda c: -len(challenges_by_champions[c]))
    
    # iterate over all champion combinations
    i = 0
    for comp in itertools.combinations(champions_, 5):
        comp_challenges = find_challenges(set(comp))
        if threshold_min <= len(comp_challenges) <= threshold_max:
            yield comp, comp_challenges

        if i > max_depth:
            return
        i+=1

In [ ]:
# find best comp by faction
factions_comps = defaultdict(list)

for faction in factions:
    c = champions_by_challenge[faction]
    comps = []
    for comp, comp_challenges in find_comp(c, threshold_min=2):
        l = comp, comp_challenges
        comps.append(l)
    
    comps.sort(key=lambda l: -len(l[-1]))
    
    best_comp_n = None
    for comp, comp_challenges in comps:
        n = len(comp_challenges)
        if best_comp_n is None:
            best_comp_n = n
        
        factions_comps[faction].append((comp, list(comp_challenges)))

    factions_comps[faction].sort(key=lambda c: -len(c[-1]))

factions_comps = dict(factions_comps)

for f, v in factions_comps.items():
    print(f, len(v))

In [ ]:
# find best comps overall
comps_ = defaultdict(list)

for comp, comp_challenges in find_comp(champions, 7, max_depth=1e8):
    key = f"Best compositions ({len(comp_challenges)} challenges)"
    if key not in comps_:
        comps_[key] = []
    comps_[key].append((comp, list(comp_challenges)))

comps_ = dict(comps_)

for f, v in comps_.items():
    print(f, len(v))

In [ ]:
json.dump(factions_comps | comps_, open("static/compositions.json", "w"))